In [1]:
import numpy as np
import pandas as pd
# settings to display all columns
pd.set_option("display.max_columns", None)
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps

from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
from statsmodels.graphics.gofplots import qqplot

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.model_selection import cross_val_score, cross_validate, learning_curve, train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.inspection import permutation_importance
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
import sklearn.metrics as metrics


from housing_crawler.analysis.ads_table_processing import get_processed_ads_table

%load_ext autoreload
%autoreload 2

# Prepare data

## Obtain data for WGs

In [2]:
df_original = get_processed_ads_table()


/home/carlo/code/chvieira2/housing_crawler/housing_crawler/utils.py:35: DtypeWarning: Columns (22,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(local_path)


===> Loaded all_encoded.csv locally
===> ads_OSM.csv saved locally


In [3]:
ads_feat_df = df_original[df_original['details_searched']==1]
# ads_feat_df = ads_feat_df[ads_feat_df['city'].isin(['Berlin', 'München', 'Hamburg', 'Stuttgart', 'Köln', 'Münster', 'Leipzig', 'Frankfurt am Main'])]
ads_feat_df = ads_feat_df.set_index('id')

## Remove duplicates if exist

In [4]:
# Number of duplicate data points
# It's very likely zero cause I already removed dulicated IDs during processing
ads_feat_df.duplicated().sum()

0

## Filter data accordingly

In [5]:
# Filter only ads that have been searched for details (search added from august on)
df_filtered = ads_feat_df.copy()
df_filtered = df_filtered[df_filtered['type_offer_simple']=='WG']
df_filtered = df_filtered[df_filtered['km_to_centroid'].notna()]
df_filtered.shape

(6465, 142)

## Drop columns I won't use for modelling

In [ ]:
# Drop unnecessary columns
df_filtered = df_filtered[[
#         'url',
        'commercial_landlord',
#         'title',
#         'type_offer_simple',
        'size_sqm',
#         'home_total_size',
#         'available_rooms',
        'capacity',
#         'available_spots_wg',
    
#         'address',
        'city',
#         'zip_code',
#         'latitude',
#         'longitude',
    
#         'published_on',
#         'published_at',
#         'day_of_week_publication',
#         'available_from',
#         'available_to',
        'days_available',
        'rental_length_term',
        'sin_published_at',
        'cos_published_at',
        'sin_day_week_int',
        'cos_day_week_int',
    
#         'details_searched',
#         'crawler',
    
# Values
        'price_euros',    
#         'cold_rent_euros',
#         'mandatory_costs_euros',
#         'extra_costs_euros',
#         'deposit',
        'transfer_costs_euros',
#         'price_per_sqm',

# Flatmates
        'male_flatmates',
        'female_flatmates',
        'diverse_flatmates',
        'min_age_flatmates',
        'max_age_flatmates',
    
# Person searched
        'gender_searched',
        'min_age_searched',
        'max_age_searched',
        'age_category_searched',
    
# Details
    
        'schufa_needed',
#         'wg_possible',
    
#         'smoking',
        'smoking_numerical',
        'building_type',
        'building_floor',
#         'furniture',
        'furniture_numerical',
#         'kitchen',
        'kitchen_numerical',
        'heating',
        'public_transport_distance',
        'parking',
    
#         'construction_year',
#         'energy_certificate',
#         'energy_usage',
#         'energy_efficiency_class',
#         'heating_energy_source',
    
        'tv_kabel',
        'tv_satellit',
    
#         'toilet',
        'shower_type_badewanne',
        'shower_type_dusche',
    
        'floor_type_dielen',
        'floor_type_parkett',
        'floor_type_laminat',
        'floor_type_teppich',
        'floor_type_fliesen',
        'floor_type_pvc',
        'floor_type_fußbodenheizung',
    
        'extras_waschmaschine',
        'extras_spuelmaschine',
        'extras_terrasse',
        'extras_balkon',
        'extras_garten',
        'extras_gartenmitbenutzung',
        'extras_keller',
        'extras_aufzug',
        'extras_haustiere',
        'extras_fahrradkeller',
        'extras_dachboden',
    
# WG only
        'number_languages',
        'languages_deutsch',
        'languages_englisch',
    
        'wg_type_studenten',
        'wg_type_keine_zweck',
        'wg_type_maenner',
        'wg_type_business',
        'wg_type_wohnheim',
        'wg_type_vegetarisch_vegan',
        'wg_type_alleinerziehende',
        'wg_type_funktionale',
        'wg_type_berufstaetigen',
        'wg_type_gemischte',
        'wg_type_mit_kindern',
        'wg_type_verbindung',
        'wg_type_lgbtqia',
        'wg_type_senioren',
        'wg_type_inklusive',
        'wg_type_wg_neugruendung',
    
        'internet_speed',
        'internet_dsl',
        'internet_wlan',
        'internet_flatrate',
    

# Geographical
        'km_to_centroid',
        'sin_degrees_to_centroid',
        'cos_degrees_to_centroid',

# OSM features
        'comfort_leisure_spots',
        'comfort_warehouse',
        'activities_education',
        'mobility_public_transport_bus',
        'activities_economic',
        'comfort_industrial',
        'activities_goverment',
        'social_life_eating',
        'comfort_comfort_spots',
        'social_life_culture',
        'activities_supermarket',
#         'activities_public_service',
        'social_life_community',
        'comfort_leisure_mass',
        'activities_educational',
        'mobility_street_secondary',
        'mobility_public_transport_rail',
        'activities_retail',
        'social_life_night_life',
        'comfort_green_natural',
        'comfort_railway',
        'mobility_bike_infraestructure',
#         'comfort_green_forests',
        'mobility_street_primary',
        'comfort_lakes',
#         'activities_health_regional',
        'activities_health_local',
        'comfort_green_space',
        'comfort_rivers',
        'activities_post',
        'comfort_green_parks',
        'comfort_street_motorway'
        ]]

## Define numerical and categorical columns

In [ ]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)

numerical_columns = numerical_columns_selector(df_filtered)
categorical_columns = categorical_columns_selector(df_filtered)
# categorical_columns.remove('city')

# Removing outliers

In [ ]:
### List of columns with manually inserted values that would possibly contain outliers and/werid values

manually_inserted_values_columns = [
    'price_euros',
    'size_sqm',
#     'cold_rent_euros',
#     'mandatory_costs_euros',
#     'extra_costs_euros',
#     'deposit',
#     'home_total_size',
    'public_transport_distance',
    'transfer_costs_euros',
    'min_age_flatmates',
    'max_age_flatmates',
    'min_age_searched',
    'max_age_searched',
#     'construction_year',
#     'energy_usage'
]

In [ ]:
### Manually look into the variable distribution to identify outliers
col = manually_inserted_values_columns[5]
df_filtered[col].hist(bins = 100);
# df_filtered[[col]].boxplot()
# plt.xlim(0,300)
plt.ylim(0,100);
col

In [ ]:
#### Manage identified outliers
# max_age_flatmates
# There are some ads with really extreme energy usage values (<18 or >80). These were removed for modelling
df_filtered['max_age_flatmates'] = [np.nan if (value!=value) or value < 18 or value >80 else value for value in list(df_filtered['max_age_flatmates'])]

# Dealing with NAs

In [ ]:
df_imputted = df_filtered.copy()

In [ ]:
# Percentage missing values per column before imputting
(df_imputted.isnull().sum().sort_values(ascending=False)/len(df_imputted)*100)[0:10]

## Imputting numerical columns

In [ ]:
num_cols_imputing = ['internet_speed',
                     'max_age_flatmates','min_age_flatmates',
                    'cos_published_at', 'sin_published_at',
                    'public_transport_distance','building_floor']

col = num_cols_imputing[0]
df_imputted[col].hist(bins = 100);
# df_filtered[[col]].boxplot()
# plt.xlim(0,300)
# plt.ylim(0,100);
col

In [ ]:
num_cols_imputing_mean = ['min_age_flatmates', 'max_age_flatmates']
num_cols_imputing_median = ['internet_speed', 'cos_published_at', 'sin_published_at','public_transport_distance', 'building_floor'] 

num_imputer_mean = SimpleImputer(strategy="mean") # Instantiate a SimpleImputer object with your strategy of choice
num_imputer_median = SimpleImputer(strategy="median") # Instantiate a SimpleImputer object with your strategy of choice

num_imputer_mean.fit(df_imputted[num_cols_imputing_mean]) # Call the "fit" method on the object
num_imputer_median.fit(df_imputted[num_cols_imputing_median]) # Call the "fit" method on the object

df_imputted[num_cols_imputing_mean] = num_imputer_mean.transform(df_imputted[num_cols_imputing_mean]) # Call the "transform" method on the object
df_imputted[num_cols_imputing_median] = num_imputer_median.transform(df_imputted[num_cols_imputing_median]) # Call the "transform" method on the object

num_imputer_mean.statistics_,  num_imputer_median.statistics_# The mean is stored in the transformer's memory

In [ ]:
# Percentage missing values per column after imputing
(df_imputted.isnull().sum().sort_values(ascending=False)/len(df_imputted)*100)[0:10]

## Imputting categorical columns

In [ ]:
# For some unknown reason imputting for transfer_costs_euros doesn't work, so I manually imput 0 values here for transfer_costs_euros without a answer
features_noanswer = ['transfer_costs_euros']

noanswer_imputer = SimpleImputer(strategy="constant", fill_value=0) 

noanswer_imputer.fit(df_imputted[features_noanswer])

df_imputted[features_noanswer] = noanswer_imputer.transform(df_imputted[features_noanswer])

In [ ]:
# Features in which NaN values represent something
features_noanswer = ['heating', 'parking', 'building_type']

noanswer_imputer = SimpleImputer(strategy="constant", fill_value="no_answer") 

noanswer_imputer.fit(df_imputted[features_noanswer])

df_imputted[features_noanswer] = noanswer_imputer.transform(df_imputted[features_noanswer])

In [ ]:
# Percentage missing values per column after imputing
(df_imputted.isnull().sum().sort_values(ascending=False)/len(df_imputted)*100)[0:10]

# Scaling columns

In [ ]:
df_scaled = df_imputted.copy()

## Find colums for each type of scaling

In [ ]:
features_OSM = [
                'comfort_leisure_spots',
                'comfort_warehouse',
                'activities_education',
                'mobility_public_transport_bus',
                'activities_economic',
                'comfort_industrial',
                'activities_goverment',
                'social_life_eating',
                'comfort_comfort_spots',
                'social_life_culture',
                'activities_supermarket',
#                 'activities_public_service',
                'social_life_community',
                'comfort_leisure_mass',
                'activities_educational',
                'mobility_street_secondary',
                'mobility_public_transport_rail',
                'activities_retail',
                'social_life_night_life',
                'comfort_green_natural',
                'comfort_railway',
                'mobility_bike_infraestructure',
#                 'comfort_green_forests',
                'mobility_street_primary',
                'comfort_lakes',
#                 'activities_health_regional',
                'activities_health_local',
                'comfort_green_space',
                'comfort_rivers',
                'activities_post',
                'comfort_green_parks',
                'comfort_street_motorway']

In [ ]:
continuous_num_cols = ['size_sqm', 'public_transport_distance', 'km_to_centroid'] #+ features_OSM

In [ ]:
# for numerical_feature in continuous_num_cols:
#     fig, ax = plt.subplots(1,3, figsize = (15,5))
    
#     ax[0].set_title(f"Distribution of {numerical_feature}")
#     sns.histplot(x = df_scaled[numerical_feature], kde = True, ax = ax[0])
    
#     ax[1].set_title(f"Boxplot of {numerical_feature}")
#     sns.boxplot(x = df_scaled[numerical_feature], ax = ax[1])
    
#     ax[2].set_title(f"QQplot of {numerical_feature}")
#     qqplot(df_scaled[numerical_feature], line='s', ax = ax[2])

In [ ]:
cols = continuous_num_cols#numerical_columns[0:3]
sns.pairplot(df_scaled.reset_index(), vars=cols);
print(cols)

In [ ]:
cols_minmax_scaler = ['capacity', 'sin_published_at', 'cos_published_at', 'transfer_costs_euros', 
                      'sin_day_week_int', 'cos_day_week_int', 'male_flatmates', 'female_flatmates', 'diverse_flatmates',
                      'min_age_flatmates', 'max_age_flatmates', 'min_age_searched', 'max_age_searched',
                      'smoking_numerical', 'building_floor', 'furniture_numerical', 'kitchen_numerical',
                      'number_languages', 'internet_speed', 'sin_degrees_to_centroid', 'cos_degrees_to_centroid'] + features_OSM
cols_standard_scaler = ['size_sqm', 'km_to_centroid']
cols_robust_scaler = ['public_transport_distance']

## Scaling features

In [ ]:
# Instanciate MinMaxScaler
minmax_scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()

# Fit scaler to data
minmax_scaler.fit(df_scaled[cols_minmax_scaler])
standard_scaler.fit(df_scaled[cols_standard_scaler])
robust_scaler.fit(df_scaled[cols_robust_scaler])

# Use scaler to transform data
df_scaled[cols_minmax_scaler] = minmax_scaler.transform(df_scaled[cols_minmax_scaler])
df_scaled[cols_standard_scaler] = standard_scaler.transform(df_scaled[cols_standard_scaler])
df_scaled[cols_robust_scaler] = robust_scaler.transform(df_scaled[cols_robust_scaler])

In [ ]:
df_scaled.describe()

# OneHot encoding

In [ ]:
ohe = OneHotEncoder(sparse=False, drop='if_binary', categories='auto')
feature_arr = ohe.fit_transform(df_filtered[categorical_columns])

## Get name of new columns and create new dataframe
feature_labels = ohe.get_feature_names_out()
feature_labels = np.array(feature_labels).ravel()
features = pd.DataFrame(feature_arr, columns=feature_labels)

In [ ]:
# Give correct indexes to feature table. Needed for concatenating
features.index = df_filtered.index
## Add new columns to dataframe
df_processed = pd.concat([df_scaled, features], axis = 1).drop(columns=categorical_columns)
df_processed.columns = [col.lower().replace(' ', '_').replace('ä','ae').replace('ö','oe').replace('ü','ue').replace('ß','ss') for col in df_processed.columns]

In [ ]:
df_processed.shape

# Minimize features

In [ ]:
df_minimal = df_processed.copy()

In [ ]:
df_minimal.shape

In [ ]:
# Define columns to be tested. Don't test the target, commercial_landlord and 'city'
cols_to_search = [col for col in df_minimal.columns if col not in ['price_euros', 'commercial_landlord']]
cols_to_search = [col for col in cols_to_search if not col.startswith('city_')]

cols_exclude = []
for col in cols_to_search:
    # How many times the most frequent val exists
    most_freq_count = list(df_minimal[col].value_counts())[0]
    
    if most_freq_count > len(df_minimal)*0.99:
        cols_exclude.append(col)

        
# Exclude all columns (except cities) with >99% of the same value (0) as it contains very little information
df_minimal = df_minimal.drop(columns=cols_exclude)
df_minimal.shape

In [ ]:
cols_exclude

# Colinearity

In [ ]:
df_analysed = df_minimal.copy()

In [ ]:
sns.set_theme(style = "whitegrid", font_scale= 1)
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(15, 10))

data_corr = df_minimal.corr()
sns.heatmap(data_corr, cmap='coolwarm', 
            annot = False, 
            annot_kws={"size": 8},
            vmin=-1, vmax=1);

In [ ]:
df_analysed = df_analysed.drop(columns=['age_category_searched_20_100',
                                        'extras_gartenmitbenutzung', 'gender_searched_egal'])

In [ ]:
data_corr = df_analysed.corr()

corr_df = data_corr.unstack().reset_index() # Unstack correlation matrix 
corr_df.columns = ['feature_1','feature_2', 'correlation'] # rename columns
# corr_df['correlation'] = -corr_df['correlation'] # Invert signal to see negative correlation
corr_df.sort_values(by="correlation",ascending=False, inplace=True) # sort by correlation
corr_df = corr_df[corr_df['feature_1'] != corr_df['feature_2']] # Remove self correlation
corr_df.head(30)

In [ ]:
# These are columns that are always removed, so I'm removing them prematurely to reduce the calculation time
df_analysed = df_analysed.drop(columns = ['internet_speed',
                    'max_age_searched',
#                     'rental_length_term_>=540days',
                    'days_available',
                    'min_age_flatmates',
                    'min_age_searched'
                  ])

In [ ]:
# Automatized Variation Inflation Factor (VIF) analysis
# Removing columns must be done one at a time because they influence each others VIF results

remove = True
cols_to_exclude = []
while remove:
    df = pd.DataFrame()
    
    selected_columns = ['price_euros'] # Ignore the targer column
    selected_columns = [col for col in df_analysed.columns.to_list() if col not in selected_columns]

    df["features"] = selected_columns

    df["vif_index"] = [vif(df_analysed[selected_columns].values, i) for i in range(df_analysed[selected_columns].shape[1])]

    df = round(df.sort_values(by="vif_index", ascending = False),2)
    
    df = df.head(1)

    if float(df.vif_index) >= 10:
        print(df)
        cols_to_exclude = cols_to_exclude + df.features.to_list()
        df_analysed = df_analysed.drop(columns = df.features)
    else:
        remove = False

cols_to_exclude

In [ ]:
# Variation Inflation Factor (VIF) analysis
from statsmodels.stats.outliers_influence import variance_inflation_factor as vif
df = pd.DataFrame()


selected_columns = [col for col in df_analysed.columns.to_list() if col not in ['price_euros']]

df["features"] = selected_columns

df["vif_index"] = [vif(df_analysed[selected_columns].values, i) for i in range(df_analysed[selected_columns].shape[1])]

round(df.sort_values(by="vif_index", ascending = False),2)[:10]

In [ ]:
df_analysed.info()

# Feature permutation

## Permutation analysis

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
y = df_analysed['price_euros']

model = LinearRegression().fit(X, y) # Fit model

In [ ]:
permutation_score = permutation_importance(model, X, y,
                                           scoring = ['r2','neg_root_mean_squared_error'],
                                           n_repeats=100, n_jobs=-1) # Perform Permutation

In [ ]:
importance_df = pd.DataFrame(np.vstack((X.columns,
                                        permutation_score['r2'].importances_mean,
                                       permutation_score['r2'].importances_std,
                                        permutation_score['neg_root_mean_squared_error'].importances_mean,
                                       permutation_score['neg_root_mean_squared_error'].importances_std)).T) # Unstack results

importance_df.columns=['feature',
                       'r2 feature importance','r2 feature importance std',
                       'RMSE feature importance','RMSE feature importance std']

In [ ]:
importance_df = importance_df.sort_values(by="r2 feature importance", ascending = False) # Order by importance
importance_df[:50]

In [ ]:
top_features = []
scores = []

for features in range(1, len(importance_df)): # Loop over the total number of features
    
    most_important_features = list(importance_df.head(features).feature) # List the name of the features in specific loop
   
    X_reduced = X[most_important_features] # Make feature set with the selected features
    
    cv_results = cross_val_score(model, X_reduced, y, cv=10) # cross validate
    
    scores.append(cv_results.mean()) # Append scores
    
    top_features.append(features)  # Append number of features

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,6))
plt.plot(top_features, scores)
plt.title('Top features used for modelling vs Scores')
# plt.xlim([0,100])
plt.ylim([-1,1])
plt.xlabel('Top features')
plt.ylabel('R2');

In [ ]:
importance_df_selected = importance_df[importance_df['r2 feature importance']>= 0.0001]
# max_score_n_features = scores.index(max(scores))
# importance_df_selected = importance_df.head(max_score_n_features)
print(importance_df_selected.shape)
importance_df_selected

In [ ]:
important_features = importance_df_selected.feature.to_list()

# Predictive Power Score

In [ ]:
# A correlation analysis that detects assimetric, also non-linear and numeric plus categorical relationships
# assimetric: ZIP predicts city but the city is a poor predictor of ZIP code
# also non-linear: uses Decision Tree to find relationships that might be linear or not
# numeric plus categorical: finds relationships also in categorical features
# https://towardsdatascience.com/rip-correlation-introducing-the-predictive-power-score-3d90808b9598

In [ ]:
pps_matrix = pps.matrix(df_analysed[important_features])

In [ ]:
sns.set_theme(style = "whitegrid", font_scale= 1)
fig, ax1 = plt.subplots(nrows=1, ncols=1, figsize=(25, 35))

matrix_df = pps_matrix[['x', 'y', 'ppscore']].pivot(columns='x', index='y', values='ppscore')
sns.heatmap(matrix_df, vmin=0, vmax=1, cmap="Blues", linewidths=0.5, annot=False, annot_kws={"size": 8});

#  Model parametrization and Learning curves

## Linear Regression

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = LinearRegression(n_jobs=-1),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

## Ridge linear Regression

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']


In [ ]:
%%time
# Instanciate model
model = Ridge()

# Hyperparameter search space
search_space = {
    'alpha': [0.1,1,10,100,1000],
    'tol': [0, 0.001,0.1,1],
    'solver': ['lsqr']# auto, 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga', 'lbfgs']
}

# Instanciate GridSearchCV
ridge_rsearch = GridSearchCV(
    model, search_space,
    n_jobs=-1, scoring='neg_root_mean_squared_error', cv=5, verbose=0)


ridge_rsearch.fit(X,y)
print(ridge_rsearch.best_params_)

In [ ]:
# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = Ridge(alpha = ridge_rsearch.best_params_['alpha'],
                                                                          tol = ridge_rsearch.best_params_['tol'],
                                                                          solver = ridge_rsearch.best_params_['solver']),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

## Lasso linear Regression

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

In [ ]:
%%time
# Instanciate model
model = Lasso()

# Hyperparameter search space
search_space = {
    'alpha': [0.1,1,10,100,1000],
    'tol': [0.1,1,10,100,1000],
    'selection': ['cyclic', 'random']
}

# Instanciate GridSearchCV
lasso_rsearch = GridSearchCV(
    model, search_space,
    n_jobs=-1, scoring='neg_root_mean_squared_error', cv=5, verbose=0)


lasso_rsearch.fit(X,y)
print(lasso_rsearch.best_params_)

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = Lasso(alpha = lasso_rsearch.best_params_['alpha'],
                                                                          tol = lasso_rsearch.best_params_['tol'],
                                                                          selection = lasso_rsearch.best_params_['selection']),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

## ElasticNet linear Regression

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

In [ ]:
%%time
# Instanciate model
model = ElasticNet()

# Hyperparameter search space
search_space = {
    'alpha': [0.01,0.1,1,10,100],
    'tol': [0.1,1,10,100],
    'l1_ratio': [0,0.3,0.6,1],
    'selection': ['cyclic', 'random']
}

# Instanciate GridSearchCV
elastic_rsearch = GridSearchCV(
    model, search_space,
    n_jobs=-1, scoring='neg_root_mean_squared_error', cv=5, verbose=0)


elastic_rsearch.fit(X,y)
print(elastic_rsearch.best_params_)

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = ElasticNet(alpha = elastic_rsearch.best_params_['alpha'],
                                                                               l1_ratio = elastic_rsearch.best_params_['l1_ratio'],
                                                                          tol = elastic_rsearch.best_params_['tol'],
                                                                          selection = elastic_rsearch.best_params_['selection']),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

## Stochastic Gradient Descend

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

In [ ]:
%%time
# Instanciate model
model = SGDRegressor()

# Hyperparameter search space
search_space = {
    'loss':['squared_error', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
    'alpha': [0.01,0.1,1],
    'penalty': ['elasticnet'],#['l1','l2','elasticnet'],
    'tol': [1,10,100],
    'l1_ratio': [0,0.3,0.6,1],
    'epsilon': [10,100,1000],
    'learning_rate': ['invscaling'],#,'constant','optimal','adaptive'],
    'eta0': [0.1], 
    'power_t': [0.25],
    'early_stopping': [True]
}

# Instanciate GridSearchCV
sgdr_rsearch = GridSearchCV(
    model, search_space,
    n_jobs=-1, scoring='neg_root_mean_squared_error', cv=5, verbose=0)


sgdr_rsearch.fit(X,y)
print(sgdr_rsearch.best_params_)

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = SGDRegressor(loss=sgdr_rsearch.best_params_['loss'],
                                                                                 penalty=sgdr_rsearch.best_params_['penalty'],
                                                                                 alpha=sgdr_rsearch.best_params_['alpha'],
                                                                                 l1_ratio=sgdr_rsearch.best_params_['l1_ratio'],
                                                                                 fit_intercept=True,
                                                                                 max_iter=1000,
                                                                                 tol=sgdr_rsearch.best_params_['tol'], 
                                                                                 shuffle=True, 
                                                                                 verbose=0, 
                                                                                 epsilon=sgdr_rsearch.best_params_['epsilon'], 
                                                                                 random_state=None, 
                                                                                 learning_rate=sgdr_rsearch.best_params_['learning_rate'],
                                                                                 eta0=sgdr_rsearch.best_params_['eta0'], 
                                                                                 power_t=sgdr_rsearch.best_params_['power_t'], 
                                                                                 early_stopping=sgdr_rsearch.best_params_['early_stopping'], 
                                                                                 validation_fraction=0.1,
                                                                                 n_iter_no_change=5,
                                                                                 warm_start=False, 
                                                                                 average=False),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

## KNN Regressor

In [ ]:
%%time
# Instanciate model
model = KNeighborsRegressor(n_jobs=-1)

# Hyperparameter search space
search_space = {
    'n_neighbors': range(5,50,5),
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'leaf_size': range(10,40,10)
}

# Instanciate GridSearchCV
knn_rsearch = GridSearchCV(
    model, search_space,
    n_jobs=-1, scoring='neg_root_mean_squared_error', cv=5, verbose=0)


knn_rsearch.fit(X,y)
print(knn_rsearch.best_params_)

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = KNeighborsRegressor(n_neighbors=knn_rsearch.best_params_['n_neighbors'],
                                                                                       weights=knn_rsearch.best_params_['weights'],
                                                                                       algorithm=knn_rsearch.best_params_['algorithm'],
                                                                                       leaf_size=knn_rsearch.best_params_['leaf_size']),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

## Support Vector Machine

In [ ]:
%%time
# Instanciate model
model = SVR()

# Hyperparameter search space
search_space = {
    'kernel': ['rbf'],#['linear','poly','sigmoid', 'rbf'],
    'degree': [2,3],
    'C': [500],#[10,100,1000],
    'tol': [0.001,0.01,0.1],
    'gamma': [0.1],#[0,0.1,1,'scale','auto'],
    'coef0': [0],#[0,0.1,1],
    'epsilon': [0.1,1,10]
}

# Instanciate GridSearchCV
svm_rsearch = GridSearchCV(
    model, search_space,
    n_jobs=-1, scoring='neg_root_mean_squared_error', cv=5, verbose=0)


svm_rsearch.fit(X,y)
print(svm_rsearch.best_params_)

In [ ]:
X = df_analysed.drop(columns=['price_euros'])
X = df_analysed[important_features]
y = df_analysed['price_euros']

# Range for training with 10 equally devided points
train_sizes_range = range(int(round(len(y)/10,-2)), 
                          len(y)-int(round(len(y)/10,-2)),
                          int(round(len(y)/10,-2)))

# Get train scores, train sizes, and validation scores using `learning_curve`, r2 score
train_sizes, train_scores, test_scores = learning_curve(estimator = SVR(kernel=svm_rsearch.best_params_['kernel'],
                                                                        degree=svm_rsearch.best_params_['degree'],
                                                                        gamma=svm_rsearch.best_params_['gamma'], 
                                                                        coef0=svm_rsearch.best_params_['coef0'], 
                                                                        tol=svm_rsearch.best_params_['tol'],
                                                                        C=svm_rsearch.best_params_['C'], 
                                                                        epsilon=svm_rsearch.best_params_['epsilon'], 
                                                                        shrinking=True,
                                                                        cache_size=200,
                                                                        verbose=False,
                                                                        max_iter=-1),
                                                              X = X, 
                                                              y = y, 
                                                              train_sizes = train_sizes_range, 
                                                              cv = 10,
                                                              scoring = 'neg_root_mean_squared_error')

# Take the mean of cross-validated train scores and validation scores
train_scores_mean = np.mean(train_scores, axis=1)
test_scores_mean = np.mean(test_scores, axis=1)

In [ ]:
# Plot the learning curves

with plt.style.context('seaborn-deep'):
    # figsize
    plt.figure(figsize=(10,6))
    # getting axes
    ax = plt.gca()
    # plotting
    ax.plot(train_sizes, train_scores_mean, label = 'Train score',color='blue', linestyle='dashed', marker='o',markerfacecolor='blue', markersize=10)
    ax.plot(train_sizes, test_scores_mean, label = 'Test score',color='orange', linestyle='dashed', marker='o',markerfacecolor='#ffc125', markersize=10)
    # more
    ax.set_title('Learning Curves', fontsize = 18)
    ax.set_xlabel('Training Size', fontsize = 14)
    ax.set_ylabel('RMSE', fontsize = 14)
    ax.grid(axis="x",linewidth=0.5)
    ax.grid(axis="y",linewidth=0.5)
    ax.legend(loc="best")
    
    plt.show()

# Building model

In [ ]:
## After all this analysis, the linearRegression model seems to outperform all other models....
## Also, analysis of the learning curves indicate that the model training is not yet saturated by number of data points, so I'd use as many as possible for training


In [ ]:
X = df_analysed[important_features]
# X = df_analysed.drop(columns=['price_euros'])

## Predicting the log of the prices instead of the prices directly improves prediction slightly
y = np.log2(df_analysed['price_euros'])

# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.4, 
                                                    random_state = 0)

# INSTANTIATING THE MODEL
model = LinearRegression(n_jobs=-1)

# model = Ridge(alpha = ridge_rsearch.best_params_['alpha'],
#               tol = ridge_rsearch.best_params_['tol'],
#               solver = ridge_rsearch.best_params_['solver'])

# model = Lasso(alpha = lasso_rsearch.best_params_['alpha'],
#               tol = lasso_rsearch.best_params_['tol'],
#               selection = lasso_rsearch.best_params_['selection'])

# model = ElasticNet(alpha = elastic_rsearch.best_params_['alpha'],
#                     l1_ratio = elastic_rsearch.best_params_['l1_ratio'],
#                     tol = elastic_rsearch.best_params_['tol'],
#                     selection = elastic_rsearch.best_params_['selection'])

# model = SGDRegressor(loss=sgdr_rsearch.best_params_['loss'],
#                      penalty=sgdr_rsearch.best_params_['penalty'],
#                      alpha=sgdr_rsearch.best_params_['alpha'],
#                      l1_ratio=sgdr_rsearch.best_params_['l1_ratio'],
#                      fit_intercept=True,
#                      max_iter=1000,
#                      tol=sgdr_rsearch.best_params_['tol'], 
#                      shuffle=True, 
#                      verbose=0, 
#                      epsilon=sgdr_rsearch.best_params_['epsilon'], 
#                      random_state=None, 
#                      learning_rate=sgdr_rsearch.best_params_['learning_rate'],
#                      eta0=sgdr_rsearch.best_params_['eta0'], 
#                      power_t=sgdr_rsearch.best_params_['power_t'], 
#                      early_stopping=sgdr_rsearch.best_params_['early_stopping'], 
#                      validation_fraction=0.1,
#                      n_iter_no_change=5,
#                      warm_start=False, 
#                      average=False)

# model = KNeighborsRegressor(n_neighbors=knn_rsearch.best_params_['n_neighbors'],
#                             weights=knn_rsearch.best_params_['weights'],
#                             algorithm=knn_rsearch.best_params_['algorithm'],
#                             leaf_size=knn_rsearch.best_params_['leaf_size'])

# model = SVR(kernel=svm_rsearch.best_params_['kernel'],
#             degree=svm_rsearch.best_params_['degree'],
#             gamma=svm_rsearch.best_params_['gamma'],
#             coef0=svm_rsearch.best_params_['coef0'], 
#             tol=svm_rsearch.best_params_['tol'], 
#             C=svm_rsearch.best_params_['C'], 
#             epsilon=svm_rsearch.best_params_['epsilon'], 
#             shrinking=True,
#             cache_size=200,
#             verbose=False,
#             max_iter=-1)

# TRAINING THE MODEL ON THE TRAINING SET
model.fit(X_train,y_train)

# EVALUATION
model.score(X_test, y_test)

In [ ]:
def regression_results(y_true, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_true, y_pred) 
    mse=metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_true, y_pred)
    r2=metrics.r2_score(y_true, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

In [ ]:
regression_results(y_true = 2**y_test, y_pred = 2**model.predict(X_test))

In [ ]:
import statsmodels.api as sm

y_pred = 2**model.predict(X_test)
y_base = [2**np.mean(y)]*len(y_pred)
y_true = 2**y_test

print(f'RMSE: {np.sqrt(((y_pred - y_true) ** 2).mean())}')
print(f'RMSE base model (mean value): {np.sqrt(((y_base - y_true) ** 2).mean())}')

n=len(y_pred)
residuals = y_true - y_pred
residuals_base = y_base - y_pred

# residuals.std()/orders.delay_vs_expected.std() * 1/(n**0.5)

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(10, 5))
sns.histplot(residuals, kde=True, edgecolor='w', ax=ax1).set(title='My model')
sns.histplot(residuals_base, kde=True, edgecolor='w', ax=ax2).set(title='Mean model')

# https://stats.stackexchange.com/questions/101274/how-to-interpret-a-qq-plot
sm.qqplot(residuals, ax=ax3)
sm.qqplot(residuals_base, ax=ax4)
plt.tight_layout()
fig.show();

In [ ]:
sns.scatterplot(x=y_pred, y=residuals);